In [2]:
datasets = {
    'All': 'data-USC00251825-USC00254335-USC00255810-USW00014935.bin',
    'Highway 30': 'data-USC00254335-USC00255810-USW00014935.bin',
    'Triangle': 'data-USC00251825-USC00254335-USC00255810.bin',
    'Columbus': 'data-USC00251825.bin',
    'Kearney': 'data-USC00254335.bin',
    'Nebraska City': 'data-USC00255810.bin',
    'Grand Island': 'data-USW00014935.bin'
}

In [ ]:
import torch
from torch import nn, optim
import torch.utils.data as Data
import os
from forecast.dataset import *
from forecast.auto import AutoDecoder

tensors = torch.load(os.path.join('data', datasets['Columbus'])).float() * 100.0

train_pct = 0.7
train_len = int(tensors.size()[0] * train_pct)
X_train = tensors[:train_len]
X_test = tensors[train_len:]

model = AutoDecoder(tensors.size()[1], 30)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

loader = Data.DataLoader(Data.TensorDataset(X_train, X_train), shuffle=True, batch_size=16)

n_epochs = 30
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 5 != 0:
        continue
    model.eval()
    with torch.no_grad():
        X_pred = model(X_train)
        train_mse = loss_fn(X_pred, X_train)
        X_pred = model(X_test)
        test_mse = loss_fn(X_pred, X_test)
    print("Epoch %d: train MSE %.4f, test MSE %.4f" % (epoch, train_mse, test_mse))

model.eval()
with torch.no_grad():
    X_pred = model(X_train)
    train_mse = loss_fn(X_pred, X_train)
    X_pred = model(X_test)
    test_mse = loss_fn(X_pred, X_test)
print("Final: train MSE %.4f, test MSE %.4f" % (epoch, train_mse, test_mse))

Epoch 0: train MSE 3.0168, test MSE 3.1042
Epoch 5: train MSE 0.5588, test MSE 0.5737
Epoch 10: train MSE 0.1400, test MSE 0.1514
Epoch 15: train MSE 0.0920, test MSE 0.1155
Epoch 20: train MSE 2.3897, test MSE 2.3890
Epoch 25: train MSE 0.1150, test MSE 0.1201


TypeError: not all arguments converted during string formatting